In [1]:
import pprint
import json
from pathlib import Path
import os


In [3]:
def read_dataset(path):
    # the file is originally json-list format
    # we want every first-level elements to be a string itself
    # for example, [{"Hi": "a'b'"}, {"Hi": "c'd'"}] -> ["""{"Hi": "a'b'"}""", """{"Hi": "c'd'"}"""]
    
    with open(path, "r", encoding="utf-8") as f:
        data = json.loads(f.read())
    
    result = [{"Input": d["Input"], "Response": json.dumps(d["Response"], ensure_ascii=False)} for d in data]
    # print(f"Read {len(result)} examples from {path}")
    # print(f"Type of result: {type(result)}")
    # print(f"Type of result[0]: {type(result[0])}")
    # print(f"Type of result[0]['Input']: {type(result[0]['Input'])}")
    # print(f"Type of result[0]['Response']: {type(result[0]['Response'])}")
    return result

import re

dataset_tr = (read_dataset("training_dataset_v2_directsql_tr.json"))
dataset_ts = (read_dataset("training_dataset_v2_directsql_ts.json"))

# Get inputs from each dataset

inputs_tr = [d["Input"] for d in dataset_tr]
inputs_ts = [d["Input"] for d in dataset_ts]

print(f"Number of training examples: {len(inputs_tr)}")
print(f"Number of test examples: {len(inputs_ts)}")

# List all the inputs in the training dataset and test dataset

print("Training dataset inputs:")
pprint.pprint(inputs_tr)

print("Test dataset inputs:")
pprint.pprint(inputs_ts)

Number of training examples: 38
Number of test examples: 17
Training dataset inputs:
['오늘 우리반과 옆반의 평균 온도차이 알려줘',
 '올해 여름 우리반 평균온도 알려줘',
 '이번달 중 우리반 온도가 가장 더운날이 언제야?',
 '우리반과 옆반중 더 추운곳은 어디야?',
 '에어컨 켜져있어?',
 '앞반 지금 전원 켜져있어?',
 '어제 전원 껐어?',
 '지금 옆반 온도랑 우리반 온도 알려줘',
 '지난주 설정온도 변화 추이 알려줘',
 '우리반의 가장 최근 설정 온도 알려줘',
 '우리반 왜이리 덥노',
 'What time is it now?',
 'Why is our classroom so cold',
 'What is the current temperature in our class?',
 '내일 실내온도 어떨거 같아?',
 '10년 전 오늘 우리반 온도는?',
 '롯데캐슬의 현재 온도 알려줘',
 '다음 대통령 선거는 언제인가요?',
 '1층 평균 실내온도 알려줘',
 '에어컨 꺼진 방들 알려줘',
 '우리반 이번달 제일 추웠던 날은 언제냐?',
 '우리반과 옆반 이번주 설정온도 차이 보여줘',
 '어제 우리반 에어컨 작동 시간 알려줘',
 '농담 좀 해줘',
 '에어켠 제일 세게 킨 방 3개 알려줘',
 '지난 4개월동안 옆반에서 가장 더웠던 날 알려줘.',
 '오늘 아침과 저녁의 온도차이는 얼마나 돼?',
 '지난달에 설정온도랑 현재온도 차이가 제일 많이 났던 날은 언제야?',
 '지난 한달간 설정온도 평균을 알려줘.',
 '현재 설정온도랑 실내온도 차이 알려줘.',
 'What is the average temperature in our class over the last month?',
 'What is the temperature difference between our class and the adjacent class '
 'right now?',
 '어제 옆반 에어

In [5]:
# Get inputs overlapping between training and test dataset

overlap = set(inputs_tr).intersection(set(inputs_ts))
print(f"Number of overlapping inputs: {len(overlap)}")
print("Overlapping inputs:")
pprint.pprint(overlap)

# Fine if responses also overlap for the overlapping inputs
responses_tr = {d["Input"]: d["Response"] for d in dataset_tr}
responses_ts = {d["Input"]: d["Response"] for d in dataset_ts}

for k in overlap:
    if responses_tr[k] != responses_ts[k]:
        print(f"Responses for input '{k}' do not match")
        print(f"Response in tr dataset: {responses_tr[k]}")
        print(f"Response in ts dataset: {responses_ts[k]}")
    else:
        print(f"Responses for input '{k}' match")

Number of overlapping inputs: 5
Overlapping inputs:
{'앞반 지금 전원 켜져있어?',
 '어제 전원 껐어?',
 '에어컨 켜져있어?',
 '우리반과 옆반중 더 추운곳은 어디야?',
 '지금 옆반 온도랑 우리반 온도 알려줘'}
Responses for input '우리반과 옆반중 더 추운곳은 어디야?' do not match
Response in training dataset: {"Formalized Input": "지금 우리반과 옆반 중 실내온도가 더 낮은 곳을 찾아줘.", "Input Semantic Parsing": {"Temporal": {"지금": "2022-09-30 12:00:00"}, "Spatial": {"우리반": "01_IB5", "옆반": "01_IB7"}, "Modality": {"실내온도": "roomtemp"}, "Operation": ["최저"]}, "Strategy": {"Objective": "현재 우리반과 옆반의 실내온도 중 가장 낮은 온도를 가진 반을 찾고, 두 온도의 차이를 계산.", "Expected Outputs": ["옆반(17.0℃)이 우리반(19.3℃)보다 2.3℃ 더 낮아요."], "Step": ["우리반의 현재 실내온도 데이터를 쿼리해 qr_roomtemp_ours에 저장", "옆반의 현재 실내온도 데이터를 쿼리해 qr_roomtemp_beside에 저장", "두 반의 실내온도를 비교하여 더 낮은 온도를 가진 반을 찾음", "각 반의 실내온도 및 차이 반환"]}, "Instruction Set": [{"type": "q", "run": {"args": {"table_name": "data_t", "columns": ["roomtemp"], "conditions": ["timestamp = '2022-09-30 12:00:00'"], "subquery": "idu_id IN (SELECT id FROM idu_t WHERE name IN ('01_IB5'))"}}, "res